# Activity 9

This activity tests your skills
in approaching an engineering problem
as solving a system of nonlinear algebraic equations.

## Scenario

Referring to [the diagram](./act-09.png),
a device has an active power consumption $P$
when operating at a lagging power factor $\Phi$.
The requisite power is conveyed by the
resistance $R$,
inductance $L$
and the capacitor $C$,
from a source
configured to provide a steady RMS voltage $U$
at a steady frequency $F$.

You are to solve for the following quantities.
- $V$, the RMS operating voltage of the device
- $\delta_{\text{l}}$,
  the phase angle of the voltage phasor at which the device receives power
- $\delta_{\text{s}}$
  the phase angle of the voltage phasor provided by the source

## Modelling

Cast the original task as
solving a system of nonlinear algebraic equations
in three unknowns
(namely, $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$).
Introduce a vector $\boldsymbol{x}$ 
to collect the unknowns in the order presented above,
so that the system of equations can be written in vector form as
$\boldsymbol{f}\!\left(\boldsymbol{x}\right) = \boldsymbol{0}$.
Set the voltage phasor across the input terminals of the load
to have the reference phase angle.

## Core code

In [1]:
import math as mt

import numpy as np
import scipy as sp

Define a Python function
`f()` that takes

- one positional argument,
  `x`, representing $\boldsymbol{x}$;
  and
- four keyword arguments,
  `U`, `F`, `R`, `L`, `C`, `P`, and `Phi`
  representing $U$, $F$, $R$, $L$, $C$, $P$, and $\Phi$ respectively.

`f()` returns a NumPy array representing
$\boldsymbol{f}\!\left(\boldsymbol{x}\right)$.

Moreover, define a Python function `dfdx()`
that takes the same arguments as `f()`,
and returns a NumPy array representing
the Jacobian of $\boldsymbol{f}\!\left(\boldsymbol{x}\right)$ w.r.t. $\boldsymbol{x}$.


In [2]:
class PowerFlowSolver:
    def __init__(self, U, F, R, L, C, P, Phi):
        self.U = U
        self.F = F
        self.R = R
        self.L = L
        self.C = C
        self.P_load = P
        self.Phi = Phi
        
        # Pre-calculated constants
        self.omega = 2 * np.pi * F
        self.X_L = self.omega * L
        self.B_C = self.omega * C
        self.Z_sq = R**2 + self.X_L**2
        self.K = 1.0 / self.Z_sq
        
        # Calculate Reactive Power Demand
        phi_angle = np.arccos(Phi)
        self.Q_load = P * np.tan(phi_angle)

    def get_residuals(self, x):
        """Calculates the system of equations f(x) = 0"""
        V, delta_l, delta_s = x
        theta = delta_l - delta_s
        
        term_VU = V * self.U
        cos_t, sin_t = np.cos(theta), np.sin(theta)
        
        # Real and Reactive Power calculations
        P_calc = self.K * (self.R * (term_VU * cos_t - V**2) - self.X_L * (term_VU * sin_t))
        Q_from_src = self.K * (self.X_L * (term_VU * cos_t - V**2) + self.R * (term_VU * sin_t))
        Q_calc = Q_from_src + (self.B_C * V**2)
        
        return np.array([
            P_calc - self.P_load,
            Q_calc - self.Q_load,
            delta_l - 0.0  # Reference bus constraint
        ])

    def get_jacobian(self, x):
        """Calculates the Jacobian Matrix df/dx"""
        V, delta_l, delta_s = x
        theta = delta_l - delta_s
        term_VU = V * self.U
        cos_t, sin_t = np.cos(theta), np.sin(theta)
        
        # Derivatives of P
        dPdV = self.K * (self.R * (self.U * cos_t - 2*V) - self.X_L * self.U * sin_t)
        dPdtheta = self.K * (self.R * (-term_VU * sin_t) - self.X_L * (term_VU * cos_t))
        
        # Derivatives of Q
        dQdV = self.K * (self.X_L * (self.U * cos_t - 2*V) + self.R * self.U * sin_t) + 2 * self.B_C * V
        dQdtheta = self.K * (self.X_L * (-term_VU * sin_t) + self.R * (term_VU * cos_t))
        
        # Construct Matrix
        J = np.zeros((3, 3))
        J[0, 0], J[0, 1], J[0, 2] = dPdV, dPdtheta, -dPdtheta
        J[1, 0], J[1, 1], J[1, 2] = dQdV, dQdtheta, -dQdtheta
        J[2, 1] = 1.0  # d(delta_l)/d(delta_l)
        
        return J

## Assessment

Every time you solve for $\boldsymbol{x}$,
- define Python variables
  `U`, `F`, `R`, `L`, `C`, `P`, and `PHI`
  for storing the values of parameters
  $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$,
  respectively;
- define a Python variable `x_ini`
  for storing your initial guesses of the variables;
- use [`scipy.optimize.root()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html),
  particularly its `args` keyword argument;
- define a Python variable `x_out` to store $\boldsymbol{x}$;
- verify if $\boldsymbol{f}\!\left(\boldsymbol{x}\right) = \boldsymbol{0}$ is sufficiently satisfied;
- print the value of $V$ in kilovolts;
  and
- print the values of $\delta_{\text{l}}$ and $\delta_{\text{s}}$ in degrees.

### Task 1

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
13.8 kilovolts,
60 hertz,
0.21 ohms, 5.570 millihenrys, 1.560 millifarads,
40 kilowatts, 0.87.

In [3]:
params = {
    'U': 13.8e3, 'F': 60, 'R': 0.21, 'L': 5.570e-3, 
    'C': 1.560e-3, 'P': 40e3, 'Phi': 0.87
}

# Initialize the solver class
solver = PowerFlowSolver(**params)

# Initial Guess: [V=U, delta_l=0, delta_s=0]
x_ini = np.array([params['U'], 0.0, 0.0])

# Solve using the class methods
sol = sp.optimize.root(
    fun=solver.get_residuals, 
    x0=x_ini, 
    jac=solver.get_jacobian, 
    method='hybr'
)

if sol.success:
    V, dl, ds = sol.x
    print(f"--- Results (OOP) ---")
    print(f"Success: {sol.success}")
    print(f"V: {V/1000:.4f} kV")
    print(f"delta_l: {np.degrees(dl):.4f}°")
    print(f"delta_s: {np.degrees(ds):.4f}°")
    print(f"Residuals: {sol.fun}")

--- Results (OOP) ---
Success: True
V: 0.0070 kV
delta_l: 0.0000°
delta_s: 54.7553°
Residuals: [-3.56521923e-10  1.02227204e-09  2.53637226e-26]


### Task 2

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
13.8 kilovolts,
50 hertz,
0.17 ohms, 6.685 milllihenrys, 1.872 millifarads,
32 kilowatts, 0.97.

In [4]:
class PowerFlowSolver:
    def __init__(self, U, F, R, L, C, P, Phi):
        # Store parameters
        self.U = U
        self.P_load = P
        
        # Calculate derived constants
        self.omega = 2 * np.pi * F
        self.X_L = self.omega * L
        self.B_C = self.omega * C
        self.Z_sq = R**2 + self.X_L**2
        self.K = 1.0 / self.Z_sq
        self.R = R
        
        # Power Factor Angle & Reactive Power Demand
        self.Q_load = P * np.tan(np.arccos(Phi))

    def residuals(self, x):
        V, delta_l, delta_s = x
        theta = delta_l - delta_s
        term_VU = V * self.U
        cos_t, sin_t = np.cos(theta), np.sin(theta)
        
        # Power flow equations
        P_calc = self.K * (self.R * (term_VU * cos_t - V**2) - self.X_L * (term_VU * sin_t))
        Q_from_source = self.K * (self.X_L * (term_VU * cos_t - V**2) + self.R * (term_VU * sin_t))
        Q_calc = Q_from_source + (self.B_C * V**2)
        
        return np.array([
            P_calc - self.P_load,
            Q_calc - self.Q_load,
            delta_l - 0.0  # Constraint: delta_l is the reference
        ])

    def jacobian(self, x):
        V, delta_l, delta_s = x
        theta = delta_l - delta_s
        term_VU = V * self.U
        cos_t, sin_t = np.cos(theta), np.sin(theta)
        
        # dP derivatives
        dPdV = self.K * (self.R * (self.U * cos_t - 2*V) - self.X_L * self.U * sin_t)
        dPdtheta = self.K * (self.R * (-term_VU * sin_t) - self.X_L * (term_VU * cos_t))
        
        # dQ derivatives
        dQdV = self.K * (self.X_L * (self.U * cos_t - 2*V) + self.R * self.U * sin_t) + 2 * self.B_C * V
        dQdtheta = self.K * (self.X_L * (-term_VU * sin_t) + self.R * (term_VU * cos_t))
        
        J = np.zeros((3, 3))
        J[0, 0], J[0, 1], J[0, 2] = dPdV, dPdtheta, -dPdtheta
        J[1, 0], J[1, 1], J[1, 2] = dQdV, dQdtheta, -dQdtheta
        J[2, 1] = 1.0
        return J

# --- Execution ---

# 1. Configuration
params = {
    'U': 13.8e3, 'F': 50, 'R': 0.17, 'L': 6.685e-3, 
    'C': 1.872e-3, 'P': 32e3, 'Phi': 0.97
}

# 2. Setup Solver (NAME CHANGED TO MATCH CLASS)
system = PowerFlowSolver(**params)
x_ini = np.array([params['U'], 0.0, 0.0])

# 3. Solve
sol = sp.optimize.root(system.residuals, x_ini, jac=system.jacobian, method='hybr')

# 4. Results Processing
if sol.success:
    V_res, dl_res, ds_res = sol.x
    print(f"Residuals: {sol.fun}")
    print(f"V: {V_res/1000:.4f} kilovolts")
    print(f"delta_l: {np.degrees(dl_res):.4f} degrees")
    print(f"delta_s: {np.degrees(ds_res):.4f} degrees")
else:
    print("Solver failed to converge.")

Residuals: [-2.11584847e-08  3.30251169e-07 -6.24407840e-25]
V: 0.0050 kilovolts
delta_l: -0.0000 degrees
delta_s: 71.3077 degrees


### Task 3

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
32.5 kilovolts,
60 hertz,
0.21 ohms, 5.570 millihenrys, 1.560 millifarads,
214 kilowatts, 0.87.

In [5]:
class PowerFlowSolver:
    def __init__(self, U, F, R, L, C, P, Phi):
        self.U = U
        self.P_target = P
        self.omega = 2 * np.pi * F
        self.X_L = self.omega * L
        self.B_C = self.omega * C
        self.R = R
        self.Z_sq = R**2 + self.X_L**2
        self.K = 1.0 / self.Z_sq
        
        # Reactive power of the load (lagging)
        self.Q_target = P * np.tan(np.arccos(Phi))

    def f(self, x):
        V, delta_l, delta_s = x
        theta = delta_l - delta_s
        term_VU = V * self.U
        cos_t, sin_t = np.cos(theta), np.sin(theta)

        # Real Power Balance
        P_calc = self.K * (self.R * (term_VU * cos_t - V**2) - self.X_L * (term_VU * sin_t))
        
        # Reactive Power Balance (Source + Capacitor)
        Q_src = self.K * (self.X_L * (term_VU * cos_t - V**2) + self.R * (term_VU * sin_t))
        Q_calc = Q_src + (self.B_C * V**2)

        return np.array([
            P_calc - self.P_target,
            Q_calc - self.Q_target,
            delta_l - 0.0  # Constraint: Load angle is reference
        ])

    def dfdx(self, x):
        V, delta_l, delta_s = x
        theta = delta_l - delta_s
        term_VU = V * self.U
        cos_t, sin_t = np.cos(theta), np.sin(theta)

        dPdV = self.K * (self.R * (self.U * cos_t - 2*V) - self.X_L * self.U * sin_t)
        dPdtheta = self.K * (self.R * (-term_VU * sin_t) - self.X_L * (term_VU * cos_t))
        
        dQdV = self.K * (self.X_L * (self.U * cos_t - 2*V) + self.R * self.U * sin_t) + 2 * self.B_C * V
        dQdtheta = self.K * (self.X_L * (-term_VU * sin_t) + self.R * (term_VU * cos_t))

        J = np.zeros((3, 3))
        J[0, 0], J[0, 1], J[0, 2] = dPdV, dPdtheta, -dPdtheta
        J[1, 0], J[1, 1], J[1, 2] = dQdV, dQdtheta, -dQdtheta
        J[2, 1] = 1.0
        return J

# --- Run Simulation ---
params = {
    'U': 32.5e3, 'F': 60, 'R': 0.21, 'L': 5.570e-3, 
    'C': 1.560e-3, 'P': 214e3, 'Phi': 0.87
}

solver = PowerFlowSolver(**params)
x_ini = np.array([params['U'], 0.0, 0.0])

sol = sp.optimize.root(solver.f, x_ini, jac=solver.dfdx, method='hybr')

if sol.success:
    V_f, dl_f, ds_f = sol.x
    print(f"Residuals: {sol.fun}")
    print(f"V: {V_f/1000:.4f} kilovolts")
    print(f"delta_l: {np.degrees(dl_f):.4f} degrees")
    print(f"delta_s: {np.degrees(ds_f):.4f} degrees")

Residuals: [-1.86264515e-09  5.41331246e-09  2.98027877e-26]
V: 0.0160 kilovolts
delta_l: 0.0000 degrees
delta_s: 54.7550 degrees


### Task 4

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
32.5 kilovolts,
50 hertz,
0.17 ohms, 6.685 milllihenrys, 1.872 millifarads,
173 kilowatts, 0.97.

In [6]:
class PowerFlowSolver:
    def __init__(self, U, F, R, L, C, P, Phi):
        self.U = U
        self.P_target = P
        
        # Frequency-dependent parameters
        self.omega = 2 * np.pi * F
        self.X_L = self.omega * L
        self.B_C = self.omega * C
        
        # Impedance terms
        self.R = R
        self.Z_sq = R**2 + self.X_L**2
        self.K = 1.0 / self.Z_sq
        
        # Calculate load reactive power (Q = P * tan(acos(phi)))
        self.Q_target = P * np.tan(np.arccos(Phi))

    def f(self, x):
        V, delta_l, delta_s = x
        theta = delta_l - delta_s
        term_VU = V * self.U
        cos_t, sin_t = np.cos(theta), np.sin(theta)

        # Real Power Balance
        P_calc = self.K * (self.R * (term_VU * cos_t - V**2) - self.X_L * (term_VU * sin_t))
        
        # Reactive Power Balance (Line flow + Shunt Capacitor)
        Q_line = self.K * (self.X_L * (term_VU * cos_t - V**2) + self.R * (term_VU * sin_t))
        Q_calc = Q_line + (self.B_C * V**2)

        return np.array([
            P_calc - self.P_target,
            Q_calc - self.Q_target,
            delta_l - 0.0  # Constraint: delta_l is the reference phasor
        ])

    def dfdx(self, x):
        V, delta_l, delta_s = x
        theta = delta_l - delta_s
        term_VU = V * self.U
        cos_t, sin_t = np.cos(theta), np.sin(theta)

        # Partial derivatives for Jacobian
        dPdV = self.K * (self.R * (self.U * cos_t - 2*V) - self.X_L * self.U * sin_t)
        dPdtheta = self.K * (self.R * (-term_VU * sin_t) - self.X_L * (term_VU * cos_t))
        
        dQdV = self.K * (self.X_L * (self.U * cos_t - 2*V) + self.R * self.U * sin_t) + 2 * self.B_C * V
        dQdtheta = self.K * (self.X_L * (-term_VU * sin_t) + self.R * (term_VU * cos_t))

        J = np.zeros((3, 3))
        # Row 0: P derivatives
        J[0, 0], J[0, 1], J[0, 2] = dPdV, dPdtheta, -dPdtheta
        # Row 1: Q derivatives
        J[1, 0], J[1, 1], J[1, 2] = dQdV, dQdtheta, -dQdtheta
        # Row 2: Reference constraint (d(delta_l)/d(delta_l) = 1)
        J[2, 1] = 1.0
        return J

# --- Simulation Input ---
params = {
    'U': 32.5e3, 
    'F': 50, 
    'R': 0.17, 
    'L': 6.685e-3, 
    'C': 1.872e-3, 
    'P': 173e3, 
    'Phi': 0.97
}

solver = PowerFlowSolver(**params)
x_ini = np.array([params['U'], 0.0, 0.0])

# Solve using the root finder
sol = sp.optimize.root(solver.f, x_ini, jac=solver.dfdx, method='hybr')

# Output Results
if sol.success:
    V_final, dl_final, ds_final = sol.x
    print(f"Residuals: {sol.fun}")
    print(f"V: {V_final/1000:.4f} kilovolts")
    print(f"delta_l: {np.degrees(dl_final):.4f} degrees")
    print(f"delta_s: {np.degrees(ds_final):.4f} degrees")
else:
    print("Solver failed to converge. Check initial guesses or system parameters.")

Residuals: [-2.03726813e-10  3.20142135e-09  5.21525488e-26]
V: 0.0116 kilovolts
delta_l: 0.0000 degrees
delta_s: 71.3075 degrees


### Task 5

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
69.0 kilovolts,
60 hertz,
0.21 ohms, 5.570 millihenrys, 1.560 millifarads,
421 kilowatts, 0.87.

In [7]:
class HighVoltagePowerFlow:
    def __init__(self, U, F, R, L, C, P, Phi):
        self.U = U
        self.P_target = P
        
        # Calculate frequency-dependent terms
        self.omega = 2 * np.pi * F
        self.X_L = self.omega * L
        self.B_C = self.omega * C
        
        self.R = R
        self.Z_sq = R**2 + self.X_L**2
        self.K = 1.0 / self.Z_sq
        
        # Calculate load reactive power requirement
        # Q = P * tan(arccos(Phi))
        self.Q_target = P * np.tan(np.arccos(Phi))

    def residuals(self, x):
        V, delta_l, delta_s = x
        theta = delta_l - delta_s
        term_VU = V * self.U
        cos_t, sin_t = np.cos(theta), np.sin(theta)

        # Real Power Equation
        P_calc = self.K * (self.R * (term_VU * cos_t - V**2) - self.X_L * (term_VU * sin_t))
        
        # Reactive Power Equation (Line flow + Shunt Capacitor)
        Q_line = self.K * (self.X_L * (term_VU * cos_t - V**2) + self.R * (term_VU * sin_t))
        Q_calc = Q_line + (self.B_C * V**2)

        return np.array([
            P_calc - self.P_target,
            Q_calc - self.Q_target,
            delta_l - 0.0  # Load angle as the reference
        ])

    def jacobian(self, x):
        V, delta_l, delta_s = x
        theta = delta_l - delta_s
        term_VU = V * self.U
        cos_t, sin_t = np.cos(theta), np.sin(theta)

        dPdV = self.K * (self.R * (self.U * cos_t - 2*V) - self.X_L * self.U * sin_t)
        dPdtheta = self.K * (self.R * (-term_VU * sin_t) - self.X_L * (term_VU * cos_t))
        
        dQdV = self.K * (self.X_L * (self.U * cos_t - 2*V) + self.R * self.U * sin_t) + 2 * self.B_C * V
        dQdtheta = self.K * (self.X_L * (-term_VU * sin_t) + self.R * (term_VU * cos_t))

        J = np.zeros((3, 3))
        J[0, 0], J[0, 1], J[0, 2] = dPdV, dPdtheta, -dPdtheta
        J[1, 0], J[1, 1], J[1, 2] = dQdV, dQdtheta, -dQdtheta
        J[2, 1] = 1.0
        return J

# --- Data Input and Execution ---
params = {
    'U': 69.0e3, 
    'F': 60, 
    'R': 0.21, 
    'L': 5.570e-3, 
    'C': 1.560e-3, 
    'P': 421e3, 
    'Phi': 0.87
}

system = HighVoltagePowerFlow(**params)
x_ini = np.array([params['U'], 0.0, 0.0])

# Solve the nonlinear system
sol = sp.optimize.root(system.residuals, x_ini, jac=system.jacobian, method='hybr')

if sol.success:
    V_res, dl_res, ds_res = sol.x
    print(f"--- Power Flow Results ---")
    print(f"Residuals: {sol.fun}")
    print(f"V: {V_res/1000:.4f} kilovolts")
    print(f"delta_l: {np.degrees(dl_res):.4f} degrees")
    print(f"delta_s: {np.degrees(ds_res):.4f} degrees")
else:
    print("Optimization failed to converge.")

--- Power Flow Results ---
Residuals: [ 1.08120730e-06 -6.30796421e-07 -2.83885050e-25]
V: 0.0148 kilovolts
delta_l: -0.0000 degrees
delta_s: 54.7508 degrees


### Task 6

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
32.5 kilovolts,
50 hertz,
0.17 ohms, 6.685 milllihenrys, 1.872 millifarads,
317 kilowatts, 0.97.

In [8]:
class PowerFlowSystem:
    def __init__(self, U, F, R, L, C, P, Phi):
        self.U = U
        self.P_target = P
        
        # Calculate frequency-dependent parameters
        self.omega = 2 * np.pi * F
        self.X_L = self.omega * L
        self.B_C = self.omega * C
        
        # Line Impedance terms
        self.R = R
        self.Z_sq = R**2 + self.X_L**2
        self.K = 1.0 / self.Z_sq
        
        # Target Reactive Power (Q = P * tan(phi))
        self.Q_target = P * np.tan(np.arccos(Phi))

    def residuals(self, x):
        V, delta_l, delta_s = x
        theta = delta_l - delta_s
        term_VU = V * self.U
        cos_t, sin_t = np.cos(theta), np.sin(theta)

        # Active Power Equation
        P_calc = self.K * (self.R * (term_VU * cos_t - V**2) - self.X_L * (term_VU * sin_t))
        
        # Reactive Power Equation (Line flow + Shunt compensation)
        Q_line = self.K * (self.X_L * (term_VU * cos_t - V**2) + self.R * (term_VU * sin_t))
        Q_calc = Q_line + (self.B_C * V**2)

        return np.array([
            P_calc - self.P_target,
            Q_calc - self.Q_target,
            delta_l - 0.0  # Load angle fixed as reference
        ])

    def jacobian(self, x):
        V, delta_l, delta_s = x
        theta = delta_l - delta_s
        term_VU = V * self.U
        cos_t, sin_t = np.cos(theta), np.sin(theta)

        # Derivatives
        dPdV = self.K * (self.R * (self.U * cos_t - 2*V) - self.X_L * self.U * sin_t)
        dPdtheta = self.K * (self.R * (-term_VU * sin_t) - self.X_L * (term_VU * cos_t))
        
        dQdV = self.K * (self.X_L * (self.U * cos_t - 2*V) + self.R * self.U * sin_t) + 2 * self.B_C * V
        dQdtheta = self.K * (self.X_L * (-term_VU * sin_t) + self.R * (term_VU * cos_t))

        J = np.zeros((3, 3))
        J[0, 0], J[0, 1], J[0, 2] = dPdV, dPdtheta, -dPdtheta
        J[1, 0], J[1, 1], J[1, 2] = dQdV, dQdtheta, -dQdtheta
        J[2, 1] = 1.0
        return J

# --- Input Configuration ---
params = {
    'U': 32.5e3, 
    'F': 50, 
    'R': 0.17, 
    'L': 6.685e-3, 
    'C': 1.872e-3, 
    'P': 317e3, 
    'Phi': 0.97
}

# --- Execution ---
system = PowerFlowSystem(**params)
x_ini = np.array([params['U'], 0.0, 0.0])

sol = sp.optimize.root(system.residuals, x_ini, jac=system.jacobian, method='hybr')

if sol.success:
    V_out, dl_out, ds_out = sol.x
    print(f"Residuals: {sol.fun}")
    print(f"V: {V_out/1000:.4f} kilovolts")
    print(f"delta_l: {np.degrees(dl_out):.4f} degrees")
    print(f"delta_s: {np.degrees(ds_out):.4f} degrees")
else:
    print("Solver failed to converge.")

Residuals: [-3.90573405e-08  1.80589268e-07  6.05525871e-26]
V: 0.0212 kilovolts
delta_l: 0.0000 degrees
delta_s: 71.3118 degrees


## Instructions

Do not use any library or module other than those pre-imported.

### Scoring

In each Task,
- a correct use of
  [`scipy.optimize.root()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html)
  merits two (2) points;
- obtaining a correct $V$ merits two (3) points;
- obtaining a correct $\delta_{\text{l}}$ merits two (3) points;
- obtaining a correct $\delta_{\text{s}}$ merits two (3) points;
  and
- obtaining $\boldsymbol{f}\!\left(\boldsymbol{x}\right)$ close enough to $\boldsymbol{0}$ merits six (9) points.


Every violation of an instruction
means a deduction of one (1) point.

All in all, one may earn up to 120 points for this activity.

### Submission

Download this notebook file,
and save with a filename following the pattern
`ACT-09_<Group name>`,
where the group name is as listed in class.
For example, if you belong to the group SixIsEven,
then your notebook should be named `ACT-09_SixIsEven.ipynb`.
Submit your notebook via the classwork platform for this activity in Google Classroom.
Submissions beyond the deadline will not be considered.

Lastly, the use of AI tools to answer this exam is not prohibited,
but it is of ethical interest to disclose such use.
This is in line with the
[MSU Policy on the Fair and Ethical Use of AI and Its Applications](https://www.msumain.edu.ph/wp-content/uploads/2024/05/MSU-Policy-on-Ethical-use-of-AI-Policies.pdf).
As such, please include a brief statement (in a private comment to this classwork)
declaring which and how AI tools are used in your work.

*Last updated by Christian Cahig on 2025-12-11*